# RNA Editing Analysis

Created By: Thomas Whisenant, Ph.D.

The analysis notebook below that is adapted from an Rscript created by Adam Mark. 

### Primary Analysis Pipeline
* Read in MAF file containing edited sites in all samples
* Run differential editing analysis
* Filter for significant differential editing events

### Annotation
* Species - Human
* Genome - hg19
* Transcript definitions - Gencode v19


In [1]:
library(data.table)
library(plyr)
library(maftools)
library(limma)
library(tidyverse)
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
seqlevels(TxDb.Hsapiens.UCSC.hg19.knownGene) <- c(paste0("chr", c(seq(1:22), "X", "Y")))



Warning message:
“package ‘data.table’ was built under R version 3.4.4”Warning message:
“package ‘maftools’ was built under R version 3.4.4”
Attaching package: ‘limma’

The following object is masked from ‘package:BiocGenerics’:

    plotMA

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
Warning message:
“package ‘ggplot2’ was built under R version 3.4.4”Warning message:
“package ‘tidyr’ was built under R version 3.4.4”Warning message:
“package ‘purrr’ was built under R version 3.4.4”Warning message:
“package ‘dplyr’ was built under R version 3.4.4”Warning message:
“package ‘stringr’ was built under R version 3.4.4”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()    masks plyr::arrange()
✖ dplyr::between()    masks data.table::between()
✖ dplyr::combine()    ma

## Wrapper function to discover differential edits by gene region

### Regional classifications
* Intergenic (IGR)
* 3' UTR
* 5' UTR
* 5' Flank
* Missense Mutation
* Silent
* Intron
* RNA
* lincRNA

### Steps performed in the function
* Read in maf file
* Subset maf data for phenotypes of interest
* Call differential editing analysis function
* Perform multiple testing correction by calculating adjusted p-values

### ** If this function is not working, perform the maf reading, subsetting, and filtering manually 


In [84]:
diffEditAnalysis <- function(maf, column_name=NULL, cond1, cond2, variant_class_filt=NULL, 
                             stand_padj=TRUE, n.cond1=NULL, n.cond2=NULL, use.universe=TRUE) {
  if(!is.null(variant_class_filt)) {
    maf <- subset(maf, Variant_Classification != variant_class_filt)
  }
  if(is.null(column_name)) {
      maf.comp <- subset(maf, Condition_code == cond1 | Condition_code == cond2)
  } else {
      maf.comp <- subset(maf, get(column_name) == cond1 | get(column_name) == cond2)      
  }
  maf.comp.uni <- unique(maf.comp)
  maf.comp.univ4padj <- unique(maf.comp.uni$Hugo_Symbol)
  if(is.null(column_name)){
      n.cond1 <- uniqueN(subset(maf.comp.uni, Condition_code==cond1)$Tumor_Sample_Barcode) 
      n.cond2 <- uniqueN(subset(maf.comp.uni, Condition_code==cond2)$Tumor_Sample_Barcode)  
  } else {
      n.cond1 <- uniqueN(subset(maf.comp.uni, get(column_name)==cond1)$Tumor_Sample_Barcode) 
      n.cond2 <- uniqueN(subset(maf.comp.uni, get(column_name)==cond2)$Tumor_Sample_Barcode)        
  }

  de.maf.comp.byve <- lapply(unique(maf.comp.uni$Variant_Classification), function(j) {
    do.call(rbind, lapply(unique(subset(maf.comp.uni, Variant_Classification==j)$Hugo_Symbol), function(i) {
      findDiffEditedGene(i, subset(maf.comp.uni, Variant_Classification==j), 
                         cond1, cond2, column_name, n.cond1, n.cond2)
      }))
  })
  names(de.maf.comp.byve) <- unique(maf.comp.uni$Variant_Classification)
  
  if(stand_padj) {
    #de.maf.comp.byve.df <- unique(do.call(rbind, de.maf.comp.byve))
    de.maf.comp.byve.df <- do.call(rbind, de.maf.comp.byve)
    if(use.universe) {
      de.maf.comp.byve.adj.df <- pAdjustWithUniverse(de.maf.comp.byve.df, maf.comp.univ4padj)
    } else {
      de.maf.comp.byve.adj.df <- pAdjustWithUniverse(de.maf.comp.byve.df)
    } 
  } else {
    if(use.universe) {
        de.maf.comp.byve.adj <- lapply(de.maf.comp.byve, function(i) pAdjustWithUniverse(i, maf.comp.univ4padj))        
    } else {
        de.maf.comp.byve.adj <- lapply(de.maf.comp.byve, function(i) pAdjustWithUniverse(i))
    }
    de.maf.comp.byve.adj.df <- unique(do.call(rbind, de.maf.comp.byve.adj))
  }
  de.maf.adj.df <- arrange(de.maf.comp.byve.adj.df, p.adjust)
  de.maf.adj.df
}


## Differential Editing function

### Steps performed in the function
* Subset unique edits for each gene in each sample
* Create a matrix of total edited sites and not edited sites  for each sample group
* Perform chi-square statistical test on the read matrix


In [4]:
findDiffEditedGene <- function(gene, maf, cond1, cond2, column_name=NULL, n.cond1=NULL, n.cond2=NULL){
  dis.gene <- subset(maf, Hugo_Symbol == gene)
  if(is.null(column_name)) {
    if(is.null(n.cond1)) {
      n.cond1 <- uniqueN(subset(dis.gene, Condition_code == cond1)$Tumor_Sample_Barcode)
    }
    if(is.null(n.cond2)) {
      n.cond2 <- uniqueN(subset(dis.gene, Condition_code == cond2)$Tumor_Sample_Barcode)
    }
    
    n.cond1.gene <- nrow(subset(dis.gene, Condition_code == cond1))
    n.cond2.gene <- nrow(subset(dis.gene, Condition_code == cond2))
  } else {
    if(is.null(n.cond1)) {
      n.cond1 <- uniqueN(subset(dis.gene, get(column_name) == cond1)$Tumor_Sample_Barcode)
    }
    if(is.null(n.cond2)) {
      n.cond2 <- uniqueN(subset(dis.gene, get(column_name) == cond2)$Tumor_Sample_Barcode)
    }
    n.cond1.gene <- nrow(subset(dis.gene, get(column_name) == cond1))
    n.cond2.gene <- nrow(subset(dis.gene, get(column_name) == cond2))
  }
  n.possible.sites <- uniqueN(dis.gene$Start_position)
  n.possible.sites.cond1 <- n.cond1*n.possible.sites
  n.possible.sites.cond2 <- n.cond2*n.possible.sites
  
  mat <- rbind(Edited=c(n.cond1.gene, n.cond2.gene), 
               notEdited=c((n.possible.sites.cond1-n.cond1.gene), 
                           (n.possible.sites.cond2 - n.cond2.gene)))
  mat[mat < 0] <- 0
  Xsq <- chisq.test(mat)
  data.frame(p.value=Xsq$p.value, chi_square_statistic=Xsq$statistic,
             Gene=gene,
             n_cond1=n.cond1.gene, n_cond2=n.cond2.gene,
             n_samples_cond1=n.cond1, n_samples_cond2=n.cond2,
             n_possible_sites_cond1=n.possible.sites.cond1, n_possible_sites_cond2=n.possible.sites.cond2)
}


## Function to adjust p-values from chi-square statistical test


In [6]:
pAdjustWithUniverse <- function(p.value.df, gene_universe=NULL){
  if(is.null(gene_universe)){
    n.other.genes <- 0
  } else {
    n.other.genes <- length(gene_universe) - nrow(p.value.df)
  }
  if(n.other.genes==0){
    universe.df <- data.frame(p.value=integer(), chi_square_statistic=integer(), Gene=character(),
                              n_cond1=integer(), n_cond2=integer(),
                              n_samples_cond1=integer(), n_samples_cond2=integer(),
                              n_possible_sites_cond1=integer(), n_possible_sites_cond2=integer())
  } else {
    universe.df <- data.frame(p.value=integer(n.other.genes) + 1, chi_square_statistic=integer(n.other.genes) + 1,
                            Gene=NA,
                            n_cond1=integer(n.other.genes), n_cond2=integer(n.other.genes),
                            n_samples_cond1=integer(n.other.genes), n_samples_cond2=integer(n.other.genes),
                            n_possible_sites_cond1=integer(n.other.genes), n_possible_sites_cond2=integer(n.other.genes))
  }
  sig.df <- rbind(p.value.df, universe.df)
  sig.df$p.adjust <- p.adjust(sig.df$p.value, method = "BH")
  sig.df <- sig.df[c("p.value", "p.adjust", "chi_square_statistic", "Gene", "n_cond1", "n_cond2", "n_samples_cond1", "n_samples_cond2", "n_possible_sites_cond1", "n_possible_sites_cond2")]
  sig.df
  #subset(arrange(sig.df, p.value), p.adjust < 0.05)
}


## Run Differential Editing Analysis

In [2]:
MPN.edit.maf <- read.csv(file="Downloads/MPN_Normal_known_novel_merged_rnaedit_sites_04142019.maf",
                         sep="\t", stringsAsFactors = FALSE)
MPN.edit.maf.Prog <- subset(MPN.edit.maf, Cell_type == "Progenitor")
MPN.edit.maf.Prog.noIGR <- subset(MPN.edit.maf.Prog, Variant_Classification != "IGR")
MPN.edit.maf.Prog.noIGR[is.na(MPN.edit.maf.Prog.noIGR)] <- ""
MPN.edit.maf.Prog.noIGR[MPN.edit.maf.Prog.noIGR$Matched_Norm_Sample_Barcode %in% "380", "Matched_Norm_Sample_Barcode"] <- "380_ACAGTG_S0"


In [85]:
MPN.Prog.diffedit_MFvsAN <- diffEditAnalysis(MPN.edit.maf.Prog.noIGR, column_name="Condition_code2", 
                                        cond1="MF", cond2="Aged_Normal",  
                                        stand_padj=TRUE, use.universe=FALSE)
MPN.Prog.diffedit_AMLvsAN <- diffEditAnalysis(MPN.edit.maf.Prog.noIGR, column_name="Condition_code2", 
                                               cond1="AML", cond2="Aged_Normal",
                                               stand_padj=TRUE, use.universe=FALSE)
MPN.Prog.diffedit_AMLvsMF <- diffEditAnalysis(MPN.edit.maf.Prog.noIGR, column_name="Condition_code2", 
                                               cond1="AML", cond2="MF",  
                                               stand_padj=TRUE, use.universe=FALSE)


Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

...

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

In [88]:
print("Differential Editing in MPN Datset: MF vs Aged Normal")
table(MPN.Prog.diffedit_MFvsAN$p.adjust < 0.005)
print("Differential Editing in MPN Datset: AML vs Aged Normal")
table(MPN.Prog.diffedit_AMLvsAN$p.adjust < 0.005)
print("Differential Editing in MPN Datset: AML vs MF")
table(MPN.Prog.diffedit_AMLvsMF$p.adjust < 0.005)

[1] "Differential Editing in MPN Datset: MF vs Aged Normal"



FALSE  TRUE 
 5364   834 

[1] "Differential Editing in MPN Datset: AML vs Aged Normal"



FALSE  TRUE 
 3439   757 

[1] "Differential Editing in MPN Datset: AML vs MF"



FALSE  TRUE 
 5375   158 

## Preform manual differential editing analysis

In [15]:
MPN.edit.maf.Prog_MFvsAN <- subset(MPN.edit.maf.Prog.noIGR, Condition_code2 == "Aged_Normal" | Condition_code2 == "MF") 
MPN.edit.maf.Prog_AMLvsAN <- subset(MPN.edit.maf.Prog.noIGR, Condition_code2 == "Aged_Normal" | Condition_code2 == "AML") 
MPN.edit.maf.Prog_AMLvsMF <- subset(MPN.edit.maf.Prog.noIGR, Condition_code2 == "AML" | Condition_code2 == "MF") 

MPN.edit.maf.Prog_MFvsAN.uni <- unique(MPN.edit.maf.Prog_MFvsAN)
MPN.edit.maf.Prog_AMLvsAN.uni <- unique(MPN.edit.maf.Prog_AMLvsAN)
MPN.edit.maf.Prog_AMLvsMF.uni <- unique(MPN.edit.maf.Prog_AMLvsMF)

MPN.edit.maf.Prog_MFvsAN_univ4padj <- unique(MPN.edit.maf.Prog_MFvsAN.uni$Hugo_Symbol)
MPN.edit.maf.Prog_AMLvsAN_univ4padj <- unique(MPN.edit.maf.Prog_AMLvsAN.uni$Hugo_Symbol)
MPN.edit.maf.Prog_AMLvsMF_univ4padj <- unique(MPN.edit.maf.Prog_AMLvsMF.uni$Hugo_Symbol)

[1] 72776    29

[1] 40319    29

[1] 53113    29

In [7]:
MPN_MFvsAN.ncond1 <- uniqueN(subset(MPN.edit.maf.Prog_MFvsAN.uni, Condition_code2=="MF")$Tumor_Sample_Barcode)
MPN_MFvsAN.ncond2 <- uniqueN(subset(MPN.edit.maf.Prog_MFvsAN.uni, Condition_code2=="Aged_Normal")$Tumor_Sample_Barcode)
MPN_MFvsAN.tot.de.maf.byve <- lapply(unique(MPN.edit.maf.Prog_MFvsAN.uni$Variant_Classification), function(j) {
  do.call(rbind, lapply(unique(subset(MPN.edit.maf.Prog_MFvsAN.uni, Variant_Classification==j)$Hugo_Symbol), function(i) {
    findDiffEditedGene(i, subset(MPN.edit.maf.Prog_MFvsAN.uni, Variant_Classification==j), 
                       column_name="Condition_code2", cond1="MF", cond2="Aged_Normal", 
                       n.cond1=MPN_MFvsAN.ncond1, n.cond2=MPN_MFvsAN.ncond2)
  }))
})
names(MPN_MFvsAN.tot.de.maf.byve) <- unique(MPN.edit.maf.Prog_MFvsAN.uni$Variant_Classification)
MPN_MFvsAN.tot.de.maf <- do.call(rbind, MPN_MFvsAN.tot.de.maf.byve)
MPN_MFvsAN.tot.de.maf.padj <- pAdjustWithUniverse(MPN_MFvsAN.tot.de.maf)
MPN_MFvsAN.tot.de.maf.padj.sig <- MPN_MFvsAN.tot.de.maf.padj[MPN_MFvsAN.tot.de.maf.padj$p.adjust < 0.05,]
MPN_MFvsAN.tot.de.maf.padj.sig$p.adjust2 <- p.adjust(MPN_MFvsAN.tot.de.maf.padj.sig$p.adjust)

MPN_AMLvsAN.ncond1 <- uniqueN(subset(MPN.edit.maf.Prog_AMLvsAN.uni, Condition_code2=="AML")$Tumor_Sample_Barcode)
MPN_AMLvsAN.ncond2 <- uniqueN(subset(MPN.edit.maf.Prog_AMLvsAN.uni, Condition_code2=="Aged_Normal")$Tumor_Sample_Barcode)
MPN_AMLvsAN.tot.de.maf.byve <- lapply(unique(MPN.edit.maf.Prog_AMLvsAN.uni$Variant_Classification), function(j) {
  do.call(rbind, lapply(unique(subset(MPN.edit.maf.Prog_AMLvsAN.uni, Variant_Classification==j)$Hugo_Symbol), function(i) {
    findDiffEditedGene(i, subset(MPN.edit.maf.Prog_AMLvsAN.uni, Variant_Classification==j), 
                       column_name="Condition_code2", cond1="AML", cond2="Aged_Normal",
                       n.cond1=MPN_AMLvsAN.ncond1, n.cond2=MPN_AMLvsAN.ncond2)
  }))
})
names(MPN_AMLvsAN.tot.de.maf.byve) <- unique(MPN.edit.maf.Prog_AMLvsAN.uni$Variant_Classification)
MPN_AMLvsAN.tot.de.maf <- do.call(rbind, MPN_AMLvsAN.tot.de.maf.byve)
MPN_AMLvsAN.tot.de.maf.padj <- pAdjustWithUniverse(MPN_AMLvsAN.tot.de.maf)
MPN_AMLvsAN.tot.de.maf.padj.sig <- MPN_AMLvsAN.tot.de.maf.padj[MPN_AMLvsAN.tot.de.maf.padj$p.adjust < 0.05,]
MPN_AMLvsAN.tot.de.maf.padj.sig$p.adjust2 <- p.adjust(MPN_AMLvsAN.tot.de.maf.padj.sig$p.adjust)

MPN_AMLvsMF.ncond1 <- uniqueN(subset(MPN.edit.maf.Prog_AMLvsMF.uni, Condition_code2=="AML")$Tumor_Sample_Barcode)
MPN_AMLvsMF.ncond2 <- uniqueN(subset(MPN.edit.maf.Prog_AMLvsMF.uni, Condition_code2=="MF")$Tumor_Sample_Barcode)
MPN_AMLvsMF.tot.de.maf.byve <- lapply(unique(MPN.edit.maf.Prog_AMLvsMF.uni$Variant_Classification), function(j) {
  do.call(rbind, lapply(unique(subset(MPN.edit.maf.Prog_AMLvsMF.uni, Variant_Classification==j)$Hugo_Symbol), function(i) {
    findDiffEditedGene(i, subset(MPN.edit.maf.Prog_AMLvsMF.uni, Variant_Classification==j), 
                       column_name="Condition_code2", cond1="AML", cond2="MF", 
                       n.cond1=MPN_AMLvsMF.ncond1, n.cond2=MPN_AMLvsMF.ncond2)
  }))
})
names(MPN_AMLvsMF.tot.de.maf.byve) <- unique(MPN.edit.maf.Prog_AMLvsMF.uni$Variant_Classification)
MPN_AMLvsMF.tot.de.maf <- do.call(rbind, MPN_AMLvsMF.tot.de.maf.byve)
MPN_AMLvsMF.tot.de.maf.padj <- pAdjustWithUniverse(MPN_AMLvsMF.tot.de.maf)
MPN_AMLvsMF.tot.de.maf.padj.sig <- MPN_AMLvsMF.tot.de.maf.padj[MPN_AMLvsMF.tot.de.maf.padj$p.adjust < 0.05,]
MPN_AMLvsMF.tot.de.maf.padj.sig$p.adjust2 <- p.adjust(MPN_AMLvsMF.tot.de.maf.padj.sig$p.adjust)


Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

...

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(mat):
“Chi-squared approximation may be incorr

In [13]:
print("Differential Editing in MPN Datset: MF vs Aged Normal")
table(MPN_MFvsAN.tot.de.maf.padj$p.adjust < 0.005)
print("Differential Editing in MPN Datset: AML vs Aged Normal")
table(MPN_AMLvsAN.tot.de.maf.padj$p.adjust < 0.005)
print("Differential Editing in MPN Datset: AML vs MF")
table(MPN_AMLvsMF.tot.de.maf.padj$p.adjust < 0.005)


FALSE  TRUE 
 5375   158 


FALSE  TRUE 
 3439   757 


FALSE  TRUE 
 5364   834 

In [ ]:
write.csv(MPN_MFvsAN.tot.de.maf.padj[MPN_MFvsAN.tot.de.maf.padj$p.adjust < 0.005,], 
          file="MPN_MFvsAN_diffeditedgenes.csv", quote=FALSE)
write.csv(MPN_AMLvsAN.tot.de.maf.padj[MPN_AMLvsAN.tot.de.maf.padj$p.adjust < 0.005,], 
          file="MPN_AMLvsAN_diffeditedgenes.csv", quote=FALSE)
write.csv(MPN_AMLvsMF.tot.de.maf.padj[MPN_AMLvsMF.tot.de.maf.padj$p.adjust < 0.005,], 
          file="MPN_AMLvsMF_diffeditedgenes.csv", quote=FALSE)